In [76]:
import requests
import json
import pandas as pd
import time
from dateutil.parser import parse
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

apikey = 'X1o3cmgN7nsBDsInu4km3XtlPD051oPC'
wordType = 'matche'
main_url = 'https://api.neople.co.kr/cy/'
player_url = 'https://api.neople.co.kr/cy/players/'
matches_url = 'https://api.neople.co.kr/cy/matches/'

In [253]:
# 함수 정리
def get_playerid(nickname):#플레이어 고유 id를 구하는 함수
    api = main_url+'players?nickname='+nickname+'&wordType='+wordType+'&apikey='+apikey
    playerdata = requests.get(api)
    playerId = playerdata.json()['rows'][0]['playerId']
    return playerId

#매치 id 가져오기
def get_matches(playerId):
    matchids = []
    gameTypeId = 'rating'
    #검색 시작일
    startDate='2022-01-01'
    endDate='2022-03-30'
    limit = 1000
    player_url = 'https://api.neople.co.kr/cy/players/'
    cc = player_url+playerId+'/matches?gameTypeId='+gameTypeId+'&apikey='+apikey+'&startDate='+startDate+'&endDate='+endDate
    playdata = requests.get(cc)
    playdata_info= playdata.json()
    while True:
        try:
            for matche in playdata_info['matches']['rows']:
                matchids.append(matche['matchId'])
            next = playdata_info['matches']['next']
            cc_next = player_url+playerId+'/matches?gameTypeId='+gameTypeId+'&apikey='+apikey+'&startDate='+startDate+'&endDate='+endDate+'&next='+next    
            playdata = requests.get(cc_next)
            playdata_info= playdata.json()
            time.sleep(0.5)
        except:
            break
        
    return matchids


def win_and_lose(matchId):#매치 id를 들고와서 승리팀 조합과,패배팀 조합으로 나눠줌
    api3=matches_url+matchId+'?&apikey='+apikey
    game = requests.get(api3)
    lose_team = []
    win_team = []
    lose_ch = []
    win_ch = []
    try:
        if game.json()['teams'][0]['result'] == 'lose':
            for lose_player in game.json()['teams'][0]['players']:
                lose_team.append(lose_player)
            for win_player in game.json()['teams'][1]['players']:
                win_team.append(win_player)
        elif game.json()['teams'][0]['result'] == 'win':
            for lose_player in game.json()['teams'][1]['players']:
                lose_team.append(lose_player)
            for win_player in game.json()['teams'][0]['players']:
                win_team.append(win_player)
        lose_ch =[]
        win_ch = []
        for i in range(0,10):
            if game.json()['players'][i]['playerId'] in lose_team:
                lose_ch.append(game.json()['players'][i]['playInfo']['characterName'])
            elif game.json()['players'][i]['playerId'] in win_team:
                win_ch.append(game.json()['players'][i]['playInfo']['characterName'])
    except:pass
    return win_ch,lose_ch

#중복매칭 데이터 방지를 위한 데이터 불러오기
def road_antdpData():
    ant_data = []
    antdpData = pd.read_csv('MatcesData/antdpData.csv')
    antdpData.drop('Unnamed: 0',axis=1,inplace=True)
    #csv를 리스트로 변환
    for v in antdpData['0']:
        ant_data.append(v)
    return ant_data

# 이번에 얻은 데이터를 중복방지 데이터에 기록하며 저장
def antdpData_save(antdpData,now_matcs):
    antdpData.extend(now_matcs)
    antdpData = pd.DataFrame(antdpData)
    antdpData.to_csv('MatcesData/antdpData.csv')
    
    


In [78]:
# nickname = '포뇨의브루스'
# wordType = 'matche'
# main_url = 'https://api.neople.co.kr/cy/'
# player_url = 'https://api.neople.co.kr/cy/players/'
# matches_url = 'https://api.neople.co.kr/cy/matches/'
# api = main_url+'players?nickname='+nickname+'&wordType='+wordType+'&apikey='+apikey
# playerdata = requests.get(api)
# playerId = playerdata.json()['rows'][0]['playerId']
# train = pd.read_csv('train.csv')
# train.drop(['Unnamed: 0'], axis = 1, inplace = True)
# # X_train = train
# # X_train.drop(['Unnamed: 0'], axis = 1, inplace = True)
# # y_train = pd.read_csv('y_train.csv')
# # y_train.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [79]:
#
playerId = get_playerid('용1현')
matchids = get_matches(playerId)
antdpData = road_antdpData()
now_matcs = []
for v in matchids:
    if v not in antdpData:
        now_matcs.append(v)

In [80]:
antdpData_save(antdpData,now_matcs)

In [81]:
antdpData

['...']

In [240]:
game

{'date': '2022-03-29 23:40',
 'gameTypeId': 'rating',
 'map': {'mapId': '105', 'name': '그랑플람 아시아 지부'},
 'teams': [{'result': 'lose',
   'players': ['e99e0ccd8265006fa23295dec8142e17',
    '4822cb8c9d965ee6cf4ecf117b594b56',
    '743f14661a54b0aa461157d782ef3c65',
    '14d697e25d1786d596c3451514608615',
    '3c3ed41120978a011ed2813fc2e53f13']},
  {'result': 'win',
   'players': ['ba918788a4de6de5d33b7d1ed93cd82b',
    '0f4f1ba2de1f2e02f4de172470327979',
    '4c36297148e56c1aec917ceed9f952c9',
    'fb7cfc3f4bc0e3d5e45568a1e4a90502',
    'a135be388b09f7406fb0e0a3a23f52a5']}],
 'players': [{'playerId': '3c3ed41120978a011ed2813fc2e53f13',
   'nickname': '뉘들이닭맛을알아',
   'map': {'mapId': '105', 'name': '그랑플람 아시아 지부'},
   'playInfo': {'random': False,
    'partyUserCount': 0,
    'partyId': 'b762303013093c43599c55f64fdcff53',
    'playTypeName': '정상',
    'characterId': 'f414d81d3be548d47d856bfcabd50bce',
    'characterName': '나이오비',
    'level': 47,
    'killCount': 2,
    'deathCount': 5,
   

In [98]:
win_ch,lose_ch = win_and_lose('faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec6db2885f593b26a552')

In [271]:
print(game['players'][2]['items'][12])
print(game['players'][2]['items'][9])

{'itemId': '95a1d8662ca5bc9980c4c7b7e7bdf7e1', 'itemName': '낙뢰 임팩트', 'slotCode': '305', 'slotName': '특수킷', 'rarityCode': '102', 'rarityName': '언커먼', 'equipSlotCode': '305', 'equipSlotName': '특수킷'}
{'itemId': 'c2d216e72195d9c9b5b336be42cc0731', 'itemName': '엑셀레이션', 'slotCode': '302', 'slotName': '가속킷', 'rarityCode': '103', 'rarityName': '레어', 'equipSlotCode': '302', 'equipSlotName': '가속킷'}


In [265]:
basic = pd.DataFrame(columns=['nickname','characterName','position','attribute1','attribute2','attribute3','team','matchId'])

In [266]:
matchId = 'faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec6db2885f593b26a552'
api3=matches_url+matchId+'?&apikey='+apikey
game = requests.get(api3).json()
for i in range(0,10):
    if game['players'][i]['playInfo']['characterName'] in lose_ch:
        ch = []
        ch.append(game['players'][i]['nickname'])
        ch.append(game['players'][i]['playInfo']['characterName'])
        ch.append(game['players'][i]['position']['name'])
        for j in range(0,3):
            if j == 0:
                ch.append(game['players'][i]['position']['attribute'][j]['name'])
            if j == 1:
                ch.append(game['players'][i]['position']['attribute'][j]['name'])
            if j == 2:
                ch.append(game['players'][i]['position']['attribute'][j]['name'])
        ch.append(0)
        ch.append(matchId)
        ch = pd.DataFrame(ch)
        ch = ch.transpose()
        ch.columns = ['nickname','characterName','position','attribute1','attribute2','attribute3','team','matchId']
        basic = pd.concat([basic,ch])
for i in range(0,10):
    if game['players'][i]['playInfo']['characterName'] in win_ch:
        ch = []
        ch.append(game['players'][i]['nickname'])
        ch.append(game['players'][i]['playInfo']['characterName'])
        ch.append(game['players'][i]['position']['name'])
        for j in range(0,3):
            if j == 0:
                ch.append(game['players'][i]['position']['attribute'][j]['name'])
            if j == 1:
                ch.append(game['players'][i]['position']['attribute'][j]['name'])
            if j == 2:
                ch.append(game['players'][i]['position']['attribute'][j]['name'])
        ch.append(1)
        ch.append(matchId)
        ch = pd.DataFrame(ch)
        ch = ch.transpose()
        ch.columns = ['nickname','characterName','position','attribute1','attribute2','attribute3','team','matchId']
        basic = pd.concat([basic,ch])

In [267]:
basic
#1은 이긴팀 0은 진팀이다.

,nickname,characterName,position,attribute1,attribute2,attribute3,team,matchId
0,뉘들이닭맛을알아,나이오비,원거리딜러,회피 사격,돌격전차,완전한 성장,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
0,악당공룡,드렉슬러,원거리딜러,완벽주의자,사냥꾼의 본능,완전한 성장,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
0,티안나게잘던짐,케니스,탱커,최후의 저항,재생의 숨결,불굴의 의지,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
0,이론개론,제레온,탱커,전투 태세,재생의 숨결,최후의 저항,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
0,리나,시바,근거리딜러,급소 가격,할인 판매,거인 사냥꾼,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
0,윤두부,레나,탱커,기민한 몸놀림,충동 구매,최후의 저항,1,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
0,방구석게임폐인,윌라드,원거리딜러,충동 구매,할인 판매,완전한 성장,1,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
0,ㅡSHINㅡ,루이스,근거리딜러,추격자의 발걸음,전장의 학살자,거인 사냥꾼,1,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
0,똥밟았네,멜빈,원거리딜러,전장의 열기,전장의 학살자,준비된 한 발,1,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
0,폴로랄프로렌,플로리안,탱커,불굴의 의지,재생의 숨결,최후의 저항,1,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...


In [164]:
basic

,characterName,position,attribute1,attribute2,attribute3,play,win


In [131]:
for i in range(0,10):
    if '3c3ed41120978a011ed2813fc2e53f13' == game['players'][i]['playerId']:
        

원거리딜러


In [127]:
for i in range(0,10):
    print(game['players'][i]['playerId']

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_21236/1715511315.py, line 2)

In [120]:
game

{'date': '2022-03-29 23:40',
 'gameTypeId': 'rating',
 'map': {'mapId': '105', 'name': '그랑플람 아시아 지부'},
 'teams': [{'result': 'lose',
   'players': ['e99e0ccd8265006fa23295dec8142e17',
    '4822cb8c9d965ee6cf4ecf117b594b56',
    '743f14661a54b0aa461157d782ef3c65',
    '14d697e25d1786d596c3451514608615',
    '3c3ed41120978a011ed2813fc2e53f13']},
  {'result': 'win',
   'players': ['ba918788a4de6de5d33b7d1ed93cd82b',
    '0f4f1ba2de1f2e02f4de172470327979',
    '4c36297148e56c1aec917ceed9f952c9',
    'fb7cfc3f4bc0e3d5e45568a1e4a90502',
    'a135be388b09f7406fb0e0a3a23f52a5']}],
 'players': [{'playerId': '3c3ed41120978a011ed2813fc2e53f13',
   'nickname': '뉘들이닭맛을알아',
   'map': {'mapId': '105', 'name': '그랑플람 아시아 지부'},
   'playInfo': {'random': False,
    'partyUserCount': 0,
    'partyId': 'b762303013093c43599c55f64fdcff53',
    'playTypeName': '정상',
    'characterId': 'f414d81d3be548d47d856bfcabd50bce',
    'characterName': '나이오비',
    'level': 47,
    'killCount': 2,
    'deathCount': 5,
   

{'name': '탱커',
 'explain': '체력 +7%, 회피 +5%',
 'attribute': [{'level': 1,
   'id': '7475ab8f9eb2823b748a90c44869ca7d',
   'name': '불굴의 의지'},
  {'level': 2, 'id': '1d49c4504c55a43f670d4d69575760d4', 'name': '재생의 숨결'},
  {'level': 3, 'id': '678bca255e575ae96aceefacaa7aee4e', 'name': '최후의 저항'}]}

In [85]:
for i in range(0,9):
    game['players'][i]['playInfo']['random'] # 랜덤 유무
    game['players'][i]['playInfo']['characterName'] #캐릭터
    game['players'][i]['playInfo']['level']#레벨
    game['players'][i]['playInfo']['killCount']#죽인 횟수
    game['players'][i]['playInfo']['deathCount']#죽은 횟수
    game['players'][i]['playInfo']['assistCount']#어시
    game['players'][i]['playInfo']['attackPoint']#준 데미지
    game['players'][i]['playInfo']['damagePoint']#받은 데미지
    game['players'][i]['playInfo']['battlePoint']#전투점수
    game['players'][i]['playInfo']['sightPoint']#시야점수
    game['players'][i]['playInfo']['getCoin']#흭득 코인
    game['players'][i]['playInfo']['spendConsumablesCoin'] # 소모품 코인
    game['players'][i]['playInfo']['sentinelKillCount'] # 센티널(중립) 코인
    game['players'][i]['playInfo']['demolisherKillCount'] # 철거반 코인
    game['players'][i]['playInfo']['playTime']
    game['players'][i]['playInfo']['minLifeTime']
    game['players'][i]['playInfo']['maxLifeTime']
    game['players'][i]['playInfo']['responseTime']


In [214]:
playerId = get_playerid('똥밟았네')
matchids = get_matches(playerId)
win = []
lose = []
for matchid in matchids:
    winer_team,loser_team = win_and_lose(matchid)
    time.sleep(0.2)
    win.append(winer_team)
    lose.append(loser_team)
test = pd.DataFrame(win,columns=['c1','c2','c3','c4','c5'])
test['win']=1
test2 = pd.DataFrame(lose,columns=['c1','c2','c3','c4','c5'])
test2['win']=0
test3 = pd.concat([test,test2],join='inner')
test3 = test3.reset_index(drop=True, inplace=False)

test3 = test3.replace('로라스',1).replace('휴톤',2).replace('루이스',3).replace('타라',4).replace('트리비아',5).replace('카인',6).replace('레나',7).replace('드렉슬러',8).replace('도일',9).replace('토마스',10).replace('나이오비',11).replace('시바',12).replace('웨슬리',13).replace('스텔라',14).replace('앨리셔',15).replace('클레어',16).replace('다이무스',17).replace('이글',18).replace('마를렌',19).replace('샬럿',20).replace('윌라드',21).replace('레이튼',22).replace('미쉘',23).replace('린',24).replace('빅터',25).replace('카를로스',26).replace('호타루',27).replace('트릭시',28).replace('히카르도',29).replace('까미유',30).replace('자네트',31).replace('피터',32).replace('아이작',33).replace('레베카',34).replace('엘리',35).replace('마틴',36).replace('브루스',37).replace('미아',38).replace('드니스',39).replace('제레온',40).replace('루시',41).replace('티엔',42).replace('하랑',43).replace('제이',44).replace('벨져',45).replace('리첼',46).replace('리사',47).replace('릭',48).replace('제키엘',49).replace('탄야',50).replace('캐럴',51).replace('라이샌더',52).replace('디아나',53).replace('클리브',54).replace('헬레나',55).replace('에바',56).replace('론',57).replace('레오노르',58).replace('시드니',59).replace('테이',60).replace('티모시',61).replace('엘프리데',62).replace('티샤',63).replace('카로슈',64).replace('라이언',65).replace('파수꾼 A',66).replace('에밀리',67).replace('플로리안',68).replace('케니스',69).replace('이사벨',70).replace('헤나투',71).replace('루드빅',72).replace('멜빈',73)
#라이샌더뒤로 루드빅을 넣어서 하나씩 밀어야함(임시)
test3 = test3.fillna(0)
# train = pd.read_csv('train.csv')
# train.drop(['Unnamed: 0'], axis = 1, inplace = True)
# train = pd.concat([train,test3],join='inner')
# train = train.reset_index(drop=True, inplace=False)
# train.to_csv('train.csv')

['faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec6db2885f593b26a552', '26d36bd380645314f6df1402b8ad8ad18eb6896d3b04e4f60085eecd78f2ffbf', 'dc5a19eac0a02dce8a57f769ea779052e052943906e0376ba11b6fb9ea4ad41b', '7d2dfab4eb7875687ed137598d84be84a5ddb437a798578f9261158d7f78aca7', '0c7f712169047ede4c17fcce87875d53b4dffb4344f6ee132587d6833c782ed8', 'df01f9d6bb9d3092c133f73732b2f6a2dd74cef25fc030b91c0ddcc7ff0680f6', '6298f1136331588a02f660c7d4e3636670e02fd52a0d3a4c377bc211a44ffb7e', '06feb75ad3de003523d2bf2b428b9b0ca37611aed669dea2b0c1875f4c02b932', '6216de0c02097f77e6b9ca59a8ec313ecdd461e93c3650d263cbe2d9abe04f21', '8d46045714ee87371d77f8ee6e310bae69aa2c07b6d6fa30b28405946ba238fd', 'ee427d245138e82321f80fc1fb74023a1740de747e6bef308e3d75427e4212d0', '84ec4441922571b4da10ddd60fa5c8e715ff174024fde087d77f1ed4dd7aaf8c', '0f87528c133a8da929e65a44d47fa0de6ec548fdb13bed9fc76eff62802e7bec', '2ac8c1cc08bc4b07a47260f8797ef76066bec27c8a904ff1a6420bcc839bc12f', '39226469eb1f17f761c9f2cb417b7649a83f5eb7138a83

KeyError: 0

In [39]:
now_matcs

['faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec6db2885f593b26a552',
 '26d36bd380645314f6df1402b8ad8ad18eb6896d3b04e4f60085eecd78f2ffbf',
 'dc5a19eac0a02dce8a57f769ea779052e052943906e0376ba11b6fb9ea4ad41b',
 '7d2dfab4eb7875687ed137598d84be84a5ddb437a798578f9261158d7f78aca7',
 '0c7f712169047ede4c17fcce87875d53b4dffb4344f6ee132587d6833c782ed8',
 'df01f9d6bb9d3092c133f73732b2f6a2dd74cef25fc030b91c0ddcc7ff0680f6',
 '6298f1136331588a02f660c7d4e3636670e02fd52a0d3a4c377bc211a44ffb7e',
 '06feb75ad3de003523d2bf2b428b9b0ca37611aed669dea2b0c1875f4c02b932',
 '6216de0c02097f77e6b9ca59a8ec313ecdd461e93c3650d263cbe2d9abe04f21',
 '8d46045714ee87371d77f8ee6e310bae69aa2c07b6d6fa30b28405946ba238fd',
 'ee427d245138e82321f80fc1fb74023a1740de747e6bef308e3d75427e4212d0',
 '84ec4441922571b4da10ddd60fa5c8e715ff174024fde087d77f1ed4dd7aaf8c',
 '0f87528c133a8da929e65a44d47fa0de6ec548fdb13bed9fc76eff62802e7bec',
 '2ac8c1cc08bc4b07a47260f8797ef76066bec27c8a904ff1a6420bcc839bc12f',
 '39226469eb1f17f761c9f2cb417b7649

In [37]:
now_matcs

['faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec6db2885f593b26a552',
 '26d36bd380645314f6df1402b8ad8ad18eb6896d3b04e4f60085eecd78f2ffbf',
 'dc5a19eac0a02dce8a57f769ea779052e052943906e0376ba11b6fb9ea4ad41b',
 '7d2dfab4eb7875687ed137598d84be84a5ddb437a798578f9261158d7f78aca7',
 '0c7f712169047ede4c17fcce87875d53b4dffb4344f6ee132587d6833c782ed8',
 'df01f9d6bb9d3092c133f73732b2f6a2dd74cef25fc030b91c0ddcc7ff0680f6',
 '6298f1136331588a02f660c7d4e3636670e02fd52a0d3a4c377bc211a44ffb7e',
 '06feb75ad3de003523d2bf2b428b9b0ca37611aed669dea2b0c1875f4c02b932',
 '6216de0c02097f77e6b9ca59a8ec313ecdd461e93c3650d263cbe2d9abe04f21',
 '8d46045714ee87371d77f8ee6e310bae69aa2c07b6d6fa30b28405946ba238fd',
 'ee427d245138e82321f80fc1fb74023a1740de747e6bef308e3d75427e4212d0',
 '84ec4441922571b4da10ddd60fa5c8e715ff174024fde087d77f1ed4dd7aaf8c',
 '0f87528c133a8da929e65a44d47fa0de6ec548fdb13bed9fc76eff62802e7bec',
 '2ac8c1cc08bc4b07a47260f8797ef76066bec27c8a904ff1a6420bcc839bc12f',
 '39226469eb1f17f761c9f2cb417b7649

In [208]:
train

,c1,c2,c3,c4,c5,win
0,9,68,8,11.0,32.0,1
1,40,57,62,13.0,14.0,1
2,57,34,62,29.0,58.0,1
3,33,11,71,43.0,2.0,1
4,38,32,9,57.0,61.0,1
...,...,...,...,...,...,...
4531,63,69,56,43.0,5.0,0
4532,49,18,63,11.0,48.0,0
4533,34,5,17,70.0,46.0,0
4534,36,62,12,46.0,64.0,0


In [209]:
X_train = train.drop('win',axis=1)
y_train = train.win

In [210]:
X_test = test3.drop('win',axis=1)
y_test = test3.win

In [211]:
import lightgbm as lgb

lgb_clf = lgb.LGBMClassifier(max_depth = 10,min_child_samples=60,subsample=0.8)
lgb_clf.fit(X_train, y_train)
y_pred = lgb_clf.predict(X_test)

In [212]:
accuracy_score(y_test,y_pred)

0.911504424778761

In [213]:
X_train = pd.concat([X_train,X_test],join='inner')
X_train = X_train.reset_index(drop=True, inplace=False)
y_train = pd.concat([y_train,y_test],join='inner')
y_train = y_train.reset_index(drop=True, inplace=False)
X_train.to_csv('X_train.csv')
y_train.to_csv('y_train.csv')

In [190]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [156]:
y_train

""
0
1
2
3
4
...
933
934
935
936
